<a href="https://colab.research.google.com/github/hercynhcheng/PythonProject/blob/main/Linebot_Server_DevTunnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前置作業

1.   註冊並登入 LINE Developer https://developers.line.biz/zh-hant/
2.   LINE官方帳號設定 https://tw.linebiz.com/login/
3.   申請GitHub帳號 https://github.com/

line-bot-sdk-python v3 說明文件：
https://github.com/line/line-bot-sdk-python/tree/master/linebot/v3/messaging/docs

In [ ]:
#@title 安裝套件
!pip install line-bot-sdk==3.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.2/758.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.0
    Uninstalling aiohttp-3.13.0:
      Successfully uninstalled aiohttp-3.13.0


In [ ]:
#@title 匯入套件與模組
import os, getpass, json
from flask import Flask, request, abort
from io import BytesIO

# LINE Message API 套件
from linebot.v3 import WebhookHandler, WebhookParser
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage, MessagingApiBlob,
    ImageMessage, StickerMessage, VideoMessage, AudioMessage, LocationMessage, TemplateMessage, QuickReply)
from linebot.v3.messaging import (QuickReplyItem,
    PostbackAction, MessageAction, DatetimePickerAction, CameraAction, CameraRollAction, LocationAction)
from linebot.v3.webhooks import MessageEvent, TextMessageContent, ImageMessageContent, LocationMessageContent
from aiohttp import web

# 影像處理套件（含矩陣）
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


#0 影像辨識模型
1.  使用Google Teachable Machine訓練影像辨識模型
    * https://teachablemachine.withgoogle.com/train
2.  下載模型壓縮檔案
3.  解壓縮獲得keras_model.h5
4.  點選Google Colab「檔案／上傳至工作階段儲存空間」上傳keras_model.h5模型檔案

In [ ]:
'''
#匯入keras影像辨識套件
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import get_file

#模型名稱
model_name = "keras_model.h5"
#分類標註名稱
class_name = ['狗', '貓']

# 修正keras_model.h5的格式問題
f = h5py.File(model_name, mode="r+")
model_config_string = f.attrs.get("model_config")
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
f.attrs.modify('model_config', model_config_string)
f.flush()
model_config_string = f.attrs.get("model_config")
assert model_config_string.find('"groups": 1,') == -1

#載入模型
model = load_model(model_name, compile=False)
model.trainable = False
model.summary()
'''

# 1 設定對外通道

## 1-1 連結GitHub帳號登入Dev Tunnel服務

In [ ]:
#@title 安裝Microsoft Dev Tunnel CLI
!curl -sL -o /usr/bin/devtunnel https://tunnelsassetsprod.blob.core.windows.net/cli/linux-x64-devtunnel
!sudo apt-get -qq update -y
!sudo apt-get -qq install -y libsecret-1-0
!export PATH=/usr/bin
!chmod +x /usr/bin/devtunnel
!devtunnel --version
# !devtunnel -h

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libsecret-common.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libsecret-common_0.20.5-2_all.deb ...
Unpacking libsecret-common (0.20.5-2) ...
Selecting previously unselected package libsecret-1-0:amd64.
Preparing to unpack .../libsecret-1-0_0.20.5-2_amd64.deb ...
Unpac

In [ ]:
#@title 連結GitHub帳號
#@markdown 點選連結，使用 GitHub 帳戶登入，並輸入顯示的8碼
!devtunnel user login -g -d

=Browse to https://github.com/login/device and enter the code: CC1F-1F71
Logged in as hercynhcheng using GitHub.


## 1-2 產生臨時公開網址

In [ ]:
#@title 設定網路服務通訊埠(port)
# 把 localhost 特定的 port 公開到網際網路上(例如底下的port是5000)
# 背景執行，並將輸出存為檔案：/content/devtunnel_out.txt
!nohup devtunnel host -p 5000 --allow-anonymous > /content/devtunnel_out.txt 2>&1 &

In [ ]:
#@title 顯示公開的連結網址，設定LINE Webhook URL
#@markdown 如果沒有公開網址，請再次執行本步驟
public_url = []
with open('/content/devtunnel_out.txt', 'r') as f:
    # print(f.read())
    lines = f.readlines()
    for one_line_text in lines:
        if "Connect via browser" in one_line_text:
            public_url = [url.strip() for url in ":".join(one_line_text.split(':')[1:]).split(',')]
if public_url != []:
    for i in public_url:
        print(i)
else:
    print("如果沒有公開網址，請再次執行本步驟")

https://rpn5nttt.use.devtunnels.ms:5000
https://rpn5nttt-5000.use.devtunnels.ms


# 2 建立LINE BOT

In [ ]:
#@title 輸入 LINE 授權令牌
print("貼上你的 LINE Channel secret: (https://developers.line.biz/console/)")
secret = getpass.getpass()
print("貼上你的 LINE Channel access token: (https://developers.line.biz/console/)")
access_token = getpass.getpass()
configuration = Configuration(access_token=access_token)

貼上你的 LINE Channel secret: (https://developers.line.biz/console/)
··········
貼上你的 LINE Channel access token: (https://developers.line.biz/console/)
··········


In [ ]:
#@title 正式LINE BOT伺服器：請至LINE Developers設定Webhook網址
app = Flask(__name__)
port = "5000"

print(f"local http://127.0.0.1:{port} exposed to:" )
for i in public_url:
    print(i)
print("選擇其中一個網址，用於設定 LINE Webhook URL")

@app.route("/", methods=['POST'])
def linebot():
    try:
        api_client = ApiClient(configuration)
        line_bot_api = MessagingApi(api_client)
        parser = WebhookParser(secret)

        # 取得接收到的內容-->事件
        signature = request.headers['X-Line-Signature']
        body = request.get_data(as_text=True)
        try:
            events = parser.parse(body, signature)
        except InvalidSignatureError:
            return web.Response(status=400, text='Invalid signature')
        print("received events =", events)

        for event in events:  # 依序處理接收到的事件
            if not isinstance(event, MessageEvent):
                continue  # 忽略不是訊息的事件
            if isinstance(event.message, TextMessageContent):  # <-- 收到文字訊息
                received_text = event.message.text  #使用者輸入之文字
                user_id =event.source.user_id       #使用者之識別碼
                reply_msg = []                      #LINE回傳之訊息串列

                print("RECEIVE TEXT :", received_text)

                #######對話實作區開始#################################################
                #解析回應文字

                #回覆內容
                reply_text = f"你說了{len(received_text)}個字"
                reply_msg += [TextMessage(text=reply_text)]
                reply_msg += [TextMessage(text=received_text)]
                #######對話實作區結束#################################################

                #執行回覆
                line_bot_api.reply_message(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=reply_msg
                    )
                )

            elif isinstance(event.message, ImageMessageContent):  # <-- 收到影像訊息
                # 接收影像資料
                api_instance = MessagingApiBlob(api_client)
                message_content = api_instance.get_message_content(event.message.id)
                image_arr = np.asarray(Image.open(BytesIO(message_content)))
                '''
                # 顯示影像於後台
                plt.figure(figsize=(3, 3))
                plt.title(f"received image {image_arr.shape}", fontsize=10)
                plt.imshow(image_arr)
                plt.axis('off')
                plt.show()
                '''
                # 儲存影像（folder為資料夾，以message.id為檔名）
                folder = 'photos'
                os.makedirs(folder, exist_ok=True)
                with open(f"{folder}/{event.message.id}.png", 'wb') as f:
                    f.write(message_content)

                '''
                ## 影像分類模型 ##
                # 影像前處理 (image preprocess)
                image_arr = image.smart_resize(image_arr, size=(224, 224))  #  shape=(224, 224, 3)
                normalized_img_arr = (image_arr.astype(np.float32) / 127.5) - 1  # value_range [-1.0:1.0]
                input_data = normalized_img_arr[np.newaxis, ...]  # shape=(1, 224, 224, 3)
                # 模型預測 (Predicts the model)
                prediction = model.predict(input_data)
                index = np.argmax(prediction)
                confidence_score = prediction[0][index]
                reply_text = f"最大機率的預測類別：{class_name[index]}, 信心值={confidence_score:.4f}"
                print("模型預測結果：", reply_text)
                '''

                #影像網址
                image_url = f"{public_url[0]}/{folder}/{event.message.id}.png"

                #回覆內容
                reply_text = "已經收到您的照片"
                reply_msg = [TextMessage(text=reply_text),
                             ImageMessage(original_content_url=image_url, preview_image_url=image_url)]
                print("RECEIVE IMG:", image_url)

                #執行回覆
                line_bot_api.reply_message(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=reply_msg
                    )
                )

            else:  # <-- 收到其他訊息
                #回覆內容
                reply_text = "你傳的不是文字或圖片呦～"
                reply_msg = [TextMessage(text=reply_text)]

                #執行回覆
                line_bot_api.reply_message(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=reply_msg
                    )
                )

    except Exception as e:
        # 如果發生錯誤，印出收到的內容
        print("[error]", body)
        print("[error]", str(e))
    # 驗證 Webhook 使用，不能省略
    return 'OK'

app.run(port=port)

local http://127.0.0.1:5000 exposed to:
https://rpn5nttt.use.devtunnels.ms:5000
https://rpn5nttt-5000.use.devtunnels.ms
選擇其中一個網址，用於設定 LINE Webhook URL
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Oct/2025 05:17:14] "POST / HTTP/1.1" 200 -


received events = []


# 3 關閉通道

In [ ]:
# kill all devtunnel processes
!ps -ef | grep devtunnel > /content/processes_out.txt

import os, signal

with open('/content/processes_out.txt', 'r') as f:
    lines = f.readlines()
    for one_line_text in lines:
        # print(one_line_text)
        columns = [test for test in one_line_text.split(' ') if test]
        print(columns)
        pid = columns[1]
        prg = columns[7]
        try:
            os.kill(int(pid), signal.SIGTERM)
        except:
            pass

['root', '1897', '1', '0', '04:34', '?', '00:00:05', 'devtunnel', 'host', '-p', '5000', '--allow-anonymous\n']
['root', '6797', '346', '0', '04:54', '?', '00:00:00', '/bin/bash', '-c', 'ps', '-ef', '|', 'grep', 'devtunnel', '>', '/content/processes_out.txt\n']
['root', '6799', '6797', '0', '04:54', '?', '00:00:00', 'grep', 'devtunnel\n']


In [ ]:
# log out devtunnel
!devtunnel user logout

=Logged out hercynhcheng.
